In [1]:
### Imports

import os
import torch
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import RandomSampler, Sampler
from torchvision import transforms, utils
import torch.nn as nn

import tensorflow as tf
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, TensorDataset, DataLoader
from typing import Iterator, Callable, List, Tuple
import torch.optim as optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random
import collections

# PENTRU MODEL - TARAM TEODORA

In [2]:
bert_used = 'readerbench/RoBERT-large'

In [3]:
BATCH_SIZE = 128

In [4]:
USED_TYPE = 1
#### 0 for TITLE
#### 1 for CONTENT

In [5]:
MAX_TITLE_LEN = 90
MAX_CONTENT_LEN = 2048

In [6]:
import random


def reset_numpy_seed(seed_value=42):
  try:
    # Set NumPy random seed
    import numpy as np
    np.random.seed(seed_value)
    print(f'NumPy random seed set with value: {seed_value}')
  except Exception as e:
    print(f'NumPy random seed was not set: {e}')
  return


def reset_tensorflow_seed(seed_value=42):
  try:
    # Set TensorFlow random seed
    import tensorflow as tf
    success = False
    # Here we have 2 different ways to set the seed
    # depending on the version of TensorFlow
    try:
      tf.random.set_seed(seed_value)
      success = True
    except Exception as e:
      pass
    try:
      tf.set_random_seed(seed_value)
      success = True
    except Exception as e:
      pass
    if success:
      print(f'TensorFlow random seed set with value: {seed_value}')
    else:
      print(f'TensorFlow random seed was not set')
  except Exception as e:
    print(f'TensorFlow random seed was not set: {e}')
  return


def reset_torch_seed(seed_value=42):
  try:
    # Set PyTorch random seed
    import torch
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
      torch.cuda.manual_seed(seed_value)
      torch.cuda.manual_seed_all(seed_value)  # if you are using multiple GPUs
    print(f'PyTorch random seed set with value: {seed_value}')
  except Exception as e:
    print(f'PyTorch random seed was not set: {e}')
  return


def set_random_seeds(seed_value=42):
  # Set Python random seed
  random.seed(seed_value)
  reset_numpy_seed(seed_value)
  reset_tensorflow_seed(seed_value)
  reset_torch_seed(seed_value)
  return


if __name__ == '__main__':
  # Set the desired seed value
  seed = 7757

  # Set random seeds
  set_random_seeds(seed)

NumPy random seed set with value: 7757
TensorFlow random seed set with value: 7757
PyTorch random seed set with value: 7757


# GATA TARAM

# Taram Iancu

In [7]:
import nltk
from nltk.stem.snowball import stopwords
nltk.download('stopwords')
import re

def preprocess(text):
    if type(text) is str:
        # Remove words with more than half numbers
        # text = re.sub(r'\b(?=\w\d)\w\d\w\b', ' ', text)

        # Correct badly hyphened words, badly hyphened measure units, and badly formatted numbers.
        text = re.sub(r'(?<=\S)-(?=\S)', '', text) # remove bad hyphens
        text = re.sub(r'(\d+)(\s)(%|°|€|¥|£|$|cm|mm|m|km|in|ft|yd|mi|g|kg|lb|oz)(?!\w)', r'\1\3', text) # correct badly hyphened measure units
        text = re.sub(r'(\d),(\d)', r'\1.\2', text) # correct badly formatted numbers

        # URLs
        text = re.sub(r'\(?\s*https?\S*\s*\)?', r' ', text)

        # Sites
        text = re.sub(r'\s*\s', r' ', text)

        text = re.sub(r'\(?\s*@\S*\s*\)?', r' Persoană ', text)

        # text = re.sub(r'@\w+', '', text)
        # text = text.replace('@', '') # remove soft hyphens
        # text = text.replace('\xad', '') # remove soft hyphens

        # Normalize dashes and other characters.
        text = text.replace('–', '-') # normalize dashes
        text = text.replace('—', '-') # normalize dashes
        text = text.replace('−', '-') # normalize dashes
        text = text.replace('‑', '-') # normalize dashes
        text = text.replace('“', '"') # normalize quotation marks
        text = text.replace('”', '"') # normalize quotation marks
        text = text.replace('„', '"') # normalize quotation marks
        text = re.sub(r'(?<=[.,;:?!])(?=[^\s])', r' ', text)

        # Reduce multiple spaces.
        text = re.sub(' +',' ',text)

        if text[0]==' ':
            text = text[1:]

        # diacritice = 'âăîțșÎĂȚȘÂ'
        # normale = 'aaitsiatsa'
        # for i in range(len(diacritice)):
        #     text = text.replace(diacritice[i], normale[i])

        return ' '.join([x for x in text.strip().split() if x not in stopwords.words('romanian')]).strip()

        return text

# numere de telefon
# hastaguri
# pic.twiter
# ++, --
#


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Gata Taram Iancu

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
read_data = pd.read_csv(
    "/content/drive/MyDrive/NLP3/train.csv")


In [ ]:
read_data['title'] = read_data['title'].apply(lambda text: preprocess(text))

In [ ]:

print(read_data[read_data['class'] == False].shape)
print(read_data[read_data['class'] == True].shape)

(25307, 4)
(45268, 4)


In [ ]:
read_data["title"][41508]

'"Gândurile Cristoiu". Interviu generalul (r) Silviu Predoiu, exSIE: Spionajul patriotism meserie murdară'

Pregatirea datelor:
- Scoatem spatii multiple
- Url, email?
- greseli gramaticale?
- non-utf8
- procente?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(read_data[['title', 'content']], read_data['class'], test_size=0.20)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [ ]:
print(df_train.shape)
print(df_test.shape)

(56460, 3)
(14115, 3)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device = ", device)

device =  cpu


In [ ]:
#Sursa: https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(bert_used)

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['class']]
        self.titles = [tokenizer(str(text),
                                 padding='max_length', max_length = MAX_TITLE_LEN, truncation=True,
                                return_tensors="pt") for text in df['title']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_title(self, idx):
        # Fetch a batch of inputs
        return self.titles[idx]

    def __getitem__(self, idx):

        batch_title = self.get_batch_title(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_title, batch_y

In [ ]:
train_dataset = Dataset(df_train) # 11 min

In [ ]:
validation_dataset = Dataset(df_test)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True)

In [ ]:
val_dataloader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True)

In [ ]:

from transformers import BertModel

class SatireClassifier(nn.Module):
    def __init__(self,
                 input_size: int,
                 hidden_size_1: int,
                 hidden_size_2: int,
                 hidden_size_3: int,
                 device: torch.device,
                 activation_fn: Callable,
                 dropout_rate: float):
        super(SatireClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_used)
        self.input_size = input_size
        self.hidden_size = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.hidden_size_3 = hidden_size_3
        self.hidden_layer_1 = nn.Linear(input_size, hidden_size_1)
        self.hidden_layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.hidden_layer_3 = nn.Linear(hidden_size_2, hidden_size_3)
        self.output_layer = nn.Linear(hidden_size_2, 1) ####### Nu fi proasta si modifica
        self.activation_fn = activation_fn
        self.dropout = nn.Dropout(p=dropout_rate)
        self.device = device

    def forward(self, x):
        x = x.to(torch.float32).to(device)
        h1 = self.activation_fn(self.hidden_layer_1(x))
        h2 = self.activation_fn(self.hidden_layer_2(h1))
        #h3 = self.activation_fn(self.hidden_layer_3(h2))
        #h = self.activation_fn(self.dropout(self.hidden_layer(x)))
        out = self.output_layer(h2)

        return out

In [ ]:
def get_num_features():
  if USED_TYPE == 0:
    return MAX_TITLE_LEN
  else: return MAX_CONTENT_LEN

In [ ]:
model = SatireClassifier(
    input_size=get_num_features(),
    hidden_size_1=256,
    hidden_size_2=512,
    hidden_size_3=512,
    device=device,
    activation_fn=nn.ReLU(),
    dropout_rate=0.2
)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
for param in model. parameters ():
  print(param)

In [ ]:
loss_crt = nn.BCEWithLogitsLoss(

)

In [ ]:
pos_weight = torch.tensor([5])
loss_crct = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

In [ ]:
loss_crt = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=3e-3)

In [ ]:
num_train_batches = len(train_dataloader)
num_val_batches = len(val_dataloader)

#epoch_loss = 0.0
train_losses, val_losses = [], []
train_predictions, val_predictions = [], []
train_labels, val_labels = [], []
train_accuracies, val_accuracies = [], []

NUM_EPOCHS = 25

for epoch_idx in range(NUM_EPOCHS):
    train_labels = []
    val_labels = []
    train_preds = []
    val_preds = []

    train_epoch_loss = 0.0
    model.train()
    for batch_content, batch_labels in tqdm(train_dataloader):
        model.zero_grad()

        batch_labels = batch_labels \
                  .type(torch.FloatTensor) \
                  .reshape((batch_labels.shape[0], 1))
        batch_labels = batch_labels.to(device)

        mask = batch_content['attention_mask'].to(device)
        input_id = batch_content['input_ids'].squeeze(1).to(device)
        # feedforward
        out = model(input_id)

        probabilities = torch.sigmoid(out.squeeze())  # Apply sigmoid activation
        batch_predictions = (probabilities >= 0.5).long()

        # compute loss
        loss = loss_crt(out, batch_labels)
        loss.backward()
        optimizer.step()
        train_epoch_loss += loss.item()
        train_labels += batch_labels.to(int).tolist()
        train_preds += batch_predictions.tolist()

    val_preds = []
    with torch.no_grad():
        model.eval()
        val_epoch_loss = 0.0
        for batch_content, batch_labels in tqdm(val_dataloader):
            batch_labels = batch_labels \
                  .type(torch.FloatTensor) \
                  .reshape((batch_labels.shape[0], 1))
            batch_labels = batch_labels.to(device)

            mask = batch_content['attention_mask'].to(device)
            input_id = batch_content['input_ids'].squeeze(1).to(device)
            # batch_size x 10
            # feedforward
            out = model(input_id)
            probabilities = torch.sigmoid(out.squeeze())  # Apply sigmoid activation
            batch_predictions = (probabilities >= 0.5).long()

            # compute loss
            loss = loss_crt(out, batch_labels)
            val_epoch_loss += loss.item()
            val_labels += batch_labels.to(int).tolist()
            val_preds += batch_predictions.tolist()

    train_epoch_loss /= num_train_batches
    val_epoch_loss /= num_val_batches
    train_acc = accuracy_score(train_labels, train_preds)
    valid_acc = accuracy_score(val_labels, val_preds)

    train_losses.append(train_epoch_loss)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(valid_acc)

    print("epoch %d, train loss=%f, val loss=%f, train acc=%f, val acc=%f" % (
        epoch_idx, train_epoch_loss, val_epoch_loss, train_acc, valid_acc
    ))

In [ ]:
%matplotlib inline
plt.plot(range(0,len(train_losses)), train_losses, 'g', label='Training loss')
plt.plot(range(0,len(train_losses)), val_losses, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(0,len(train_accuracies)), train_accuracies, 'g', label='Training accuracy')
plt.plot(range(0,len(train_accuracies)), val_accuracies, 'b', label='Validation accuracy')
plt.title('Training and Validation accuracies')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
def compute_confusion_matrix(predictions: List[int], labels:List[int]) -> Tuple[int]:
    """
    Compute the confusion matrix.
    Arguments:
        predictions: list of model predictions (0 for cats or 1 for dogs)
        labels: list of ground truth labels (0 or 1)
    """
    correct_cats = 0
    correct_dogs = 0
    wrong_cats = 0
    wrong_dogs = 0
    for (p,l) in zip(predictions, labels):
        if p == 0 and l == 0:
            correct_cats += 1
        elif p == 0 and l == 1:
            wrong_cats += 1
        elif p == 1 and l == 1:
            correct_dogs += 1
        else:
            wrong_dogs += 1

    return (correct_cats, wrong_cats, correct_dogs, wrong_dogs)

In [ ]:
def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
     - the predictions made by the model
     - the labels
    """
    model.eval()
    epoch_loss = 0.0
    num_batches = len(val_dataloader)
    predictions = []
    labels = []
    with torch.no_grad():
        for batch_content, batch_labels in tqdm(val_dataloader):
            batch_labels = batch_labels \
                  .type(torch.FloatTensor) \
                  .reshape((batch_labels.shape[0], 1))
            batch_labels = batch_labels.to(device)

            mask = batch_content['attention_mask'].to(device)
            input_id = batch_content['input_ids'].squeeze(1).to(device)
            # batch_size x 10
            # feedforward
            out = model(input_id)
            probabilities = torch.sigmoid(out.squeeze())  # Apply sigmoid activation
            batch_predictions = (probabilities >= 0.5).long()

            predictions += batch_predictions.squeeze().tolist()
            labels += batch_labels.squeeze().tolist()

            loss = loss_crt(out, batch_labels)
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

    epoch_loss = epoch_loss/num_batches

    return epoch_loss, predictions, labels

In [ ]:
fin_loss, preds, labels = eval_epoch(
    model, val_dataloader, loss_crt, device)

In [ ]:
len(preds)

In [ ]:
fin_loss, preds, labels = eval_epoch(
    model, val_dataloader, loss_crt, device)


correct_cats, wrong_cats, correct_dogs, wrong_dogs = compute_confusion_matrix(
    preds,
    labels
)
cf_matrix = [
    [correct_cats, wrong_cats],
    [wrong_dogs, correct_dogs]
]
import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='d')

ax.set_title('Confusion Matrix\n\n');
ax.set_xlabel('Actual Values')
ax.set_ylabel('Predicted Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['0','1'])
ax.yaxis.set_ticklabels(['0','1'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
len(preds)

In [ ]:
model1 = model

In [ ]:
fin_loss, val_predictions, val_labels = eval_epoch(
    model1, val_dataloader, loss_crt, device)

In [ ]:
model2 = model ##### overfit nebun, 512, 1024, 1025 ---//----

In [ ]:
model3 = model ###### 12 epoci, 256, 256, 256,      ...... , 20 376....

In [ ]:
print(torch.bincount(y_train))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, r2_score

print(f1_score(labels, preds))

print(recall_score(labels, preds))

print(precision_score(labels, preds))

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/NLP3/test.csv')
test_data['class'] = [True for _ in range(len(test_data['title']))]
print(test_data.head())
# print(df_test.head())


print(len(test_data))





In [ ]:
from sklearn.metrics import balanced_accuracy_score

def balanced_accuracy_evaluate(model, test_data):
    final_ans = []


    fin_loss, preds, labels = eval_epoch(
    model, test_dataloader, loss_crt, device)
    return preds, labels

In [ ]:
test_data['title'] = test_data['title'].apply(lambda text: preprocess(text))

In [ ]:
test = Dataset(test_data)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE)

predict, labels = balanced_accuracy_evaluate(model, test)

print(len(predict))

In [ ]:
def code_to_text(code):
  labels = [False, True]
  return labels[code]




y_pred = []
output = predict
for i in range(len(output)):
  try:
    item0, item1 = output[i]
    y_pred.extend([code_to_text(item0), code_to_text(item1)])
  except:
    item0 = output[i]
    y_pred.append(code_to_text(item0))
predict = y_pred


i = range(len(predict))
dict = {'id':i, 'class':predict}

final = pd.DataFrame(dict)

fc = final.columns[0]
final = final.drop([fc], axis=1)
# final.columns = ['Id', 'Label']
final.to_csv('final.csv')